In [17]:
import keras
from keras import layers
import numpy as np
import random
import io


In [18]:
path = "text.txt"
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
text = text.replace("\n", " ")  # Eliminar las nuevas lineas
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Cortar el texto en bloques iguales
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype="bool")
y = np.zeros((len(sentences), len(chars)), dtype="bool")
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Corpus length: 2188
Total chars: 43
Number of sequences: 716


In [19]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)

model.summary()

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        88,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 43)             │         5,547 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,611 (365.67 KB)

 Trainable params: 93,611 (365.67 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
def sample(preds, temperature=1.0):
    # Funcion que permite muestrear 
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [5]:
epochs = 100
batch_size = 32

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    print()
    print("------- GENERADO DESPUÉS DE LA ÉPOCA: %d -------------------------------------------------------------------------------------------------------------------------------------------------" % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("\tDIVERSIDAD: ", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('\t   Generando con semilla: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("\t   Generado --> ", generated)
        print("\t--------------------------------------------------------------------------------------------------------------------------------------------------------------------------")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 3.3791

------- GENERADO DESPUÉS DE LA ÉPOCA: 0 -------------------------------------------------------------------------------------------------------------------------------------------------
	DIVERSIDAD:  0.2
	   Generando con semilla: "educación de nivel superior, brinda a la"
	   Generado -->   a de  da d ad  na   cn  d d a c  a a   a n  d aa    da n n  e d i nda  a      ar d n  da c d id       a a    a a   a   da    nnllad ral  d  a  ada ar d  a    dda  d dn a a eden a    a d l d     a  rn na d a    d nac and daa n d   nd     a a   a ada n aa  da     da a  a l   ar   daa d ada  daa ad an  l la   d  e a ddaa   a l   aadn   a  d ndad   n and da c dd  a  dads    d  da l      n   d ra    n
	--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
	DIVERSIDAD:  0.5
	   Generando con semilla: "educación de nivel superior, brind

In [21]:
# Guardar el modelo completo (arquitectura + pesos)
#model.save('modelo.keras')

# Cargar el modelo guardado en formato .keras
from keras.models import load_model

modelo_cargado = load_model('modelo.keras')

In [23]:

def generar(seed_text, length=400, temperature=1.0):
    generated = ""
    sentence = seed_text[-maxlen:]  
    for _ in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            if char in char_indices:
                x_pred[0, t, char_indices[char]] = 1.0
        preds = modelo_cargado.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char
    return generated

In [29]:
semilla = "er la demanda de educación superior de l"
print(generar(semilla, length=150, temperature=0.2))

a ingeniería en uesiy peras síocnales y cúminte estada, ylofestinder axadesionicia ta for comples y coiónde  comoluería entriciamas a tovo co con cor 
